In [1]:
! pip install openpyxl pandas numpy

Looking in indexes: https://artifactory.tcsbank.ru/artifactory/api/pypi/python-all/simple

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np

channels = pd.read_excel("../data/tg_channels.xlsx")
#df = pd.read_csv("../data/cleaned_news_exp.csv")[["message_id", "id_channel", "message", "date", "topic"]]
df = pd.read_parquet("../data/tg_news_full.parquet")[["message_id", "id_channel", "message", "date"]]

/home/mlcore/conda/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
ch_map = (channels[["id", "name"]]
          .dropna()
          .assign(id=lambda x: pd.to_numeric(x["id"], errors="coerce"))
          .dropna(subset=["id"])
          .assign(id=lambda x: x["id"].astype(int))
          .set_index("id")["name"]
          .to_dict())

df = df.copy()
df["id_channel"] = pd.to_numeric(df["id_channel"], errors="coerce")
df["channel_name"] = df["id_channel"].map(ch_map).fillna(df["id_channel"].astype("Int64").astype(str))
df.head()

,message_id,id_channel,message,date,channel_name
0,275548,3,"Правительство Словакии обсудит меры, которые м...",2025-01-02 17:00:02,Риа Новости
1,275547,3,В ДТП с туристическим автобусом в Таиланде пос...,2025-01-02 16:40:53,Риа Новости
2,275546,3,Премьер Израиля Нетаньяху выписан из больницы ...,2025-01-02 16:20:12,Риа Новости
3,275545,3,Подозреваемый в подрыве автомобиля Tesla Cyber...,2025-01-02 15:54:29,Риа Новости
4,275543,3,Спецоперация. Обстановка и главные события на ...,2025-01-02 15:32:55,Риа Новости


In [4]:
import re
import numpy as np
import pandas as pd

def clean_news_text(t: str) -> str:
    t = t or ""
    t = re.sub(r"#\w+", " ", t)
    t = re.sub(r"[⚡️📈📉🇷🇺✅❗️🔥⬛ ⬜ ⚫ ⚪🔹]+", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t

def ensure_datetime(df: pd.DataFrame, col: str = "date") -> pd.DataFrame:
    d = df.copy()
    d[col] = pd.to_datetime(d[col], utc=True, errors="coerce")
    d = d.dropna(subset=[col])
    d["date_day"] = d[col].dt.floor("D")
    return d

df = ensure_datetime(df, "date")
df["message_id"] = df["message_id"].astype(str)
df["message"] = df["message"].fillna("").astype(str).map(clean_news_text)

In [5]:
df.head()

,message_id,id_channel,message,date,channel_name,date_day
0,275548,3,"Правительство Словакии обсудит меры, которые м...",2025-01-02 17:00:02+00:00,Риа Новости,2025-01-02 00:00:00+00:00
1,275547,3,В ДТП с туристическим автобусом в Таиланде пос...,2025-01-02 16:40:53+00:00,Риа Новости,2025-01-02 00:00:00+00:00
2,275546,3,Премьер Израиля Нетаньяху выписан из больницы ...,2025-01-02 16:20:12+00:00,Риа Новости,2025-01-02 00:00:00+00:00
3,275545,3,Подозреваемый в подрыве автомобиля Tesla Cyber...,2025-01-02 15:54:29+00:00,Риа Новости,2025-01-02 00:00:00+00:00
4,275543,3,Спецоперация. Обстановка и главные события на ...,2025-01-02 15:32:55+00:00,Риа Новости,2025-01-02 00:00:00+00:00


In [6]:
#! pip install rank_bm25

In [7]:
from rank_bm25 import BM25Okapi

def tokenize_ru(text: str):
    text = text.lower()
    text = re.sub(r"[^0-9a-zа-яё\s]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.split()

corpus_tok = [tokenize_ru(t) for t in df["message"].tolist()]
bm25 = BM25Okapi(corpus_tok)

In [8]:
#! pip install faiss-cpu faiss-gpu-cu12 faiss-gpu-cu11

In [9]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/multilingual-e5-large" 
encoder = SentenceTransformer(MODEL_NAME, device="cuda")  
texts = df["message"].tolist()
doc_inputs = ["passage: " + t for t in texts]

In [10]:
# E_docs = encoder.encode(
#     doc_inputs,
#     batch_size=64,
#     show_progress_bar=True,
#     normalize_embeddings=True,
# ).astype(np.float32)

In [11]:
# import faiss

# dim = E_docs.shape[1]
# index = faiss.IndexFlatIP(dim)      
# index.add(E_docs)

In [12]:
# # save
# from pathlib import Path
# import numpy as np
# import pandas as pd
# import faiss
# import pickle

# OUT = Path("indexes")
# OUT.mkdir(parents=True, exist_ok=True)

# #rowmap
# rowmap = df[["message_id","date","date_day","id_channel","channel_name"]].copy()
# rowmap.to_parquet(OUT / "rowmap.parquet", index=False)

# np.save(OUT / "E_docs_e5_large.npy", E_docs)

# faiss.write_index(index, str(OUT / "faiss_e5_large.index"))

# with open(OUT / "bm25_corpus_tok.pkl", "wb") as f:
#     pickle.dump(corpus_tok, f)


In [13]:
# load
from pathlib import Path
import numpy as np
import pandas as pd
import faiss
import pickle

INP = Path("indexes")

rowmap = pd.read_parquet(INP / "rowmap.parquet")

E_docs = np.load(INP / "E_docs_e5_large.npy")
index = faiss.read_index(str(INP / "faiss_e5_large.index"))

with open(INP / "bm25_corpus_tok.pkl", "rb") as f:
    corpus_tok = pickle.load(f)

from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(corpus_tok)


In [14]:
assert len(rowmap) == E_docs.shape[0]
assert len(rowmap) == index.ntotal

In [15]:
import numpy as np
import pandas as pd
import re

def snippet(t: str, n: int = 1000) -> str:
    return t[:n]

def _topk_indices_from_scores(scores: np.ndarray, k: int) -> np.ndarray:
    k = min(k, len(scores))
    if k <= 0:
        return np.array([], dtype=int)
    if k == len(scores):
        idx = np.argsort(-scores)
    else:
        idx = np.argpartition(-scores, k-1)[:k]
        idx = idx[np.argsort(-scores[idx])]
    return idx.astype(int)

def dense_candidates_faiss(index, encoder, query: str, topN: int = 500):
    qv = encoder.encode(["query: " + query], normalize_embeddings=True, show_progress_bar=False).astype(np.float32)
    scores, idx = index.search(qv, topN) 
    return idx[0].astype(int), scores[0].astype(np.float32)


def hybrid_retrieve_rrf(
    df: pd.DataFrame,
    index,
    encoder,
    bm25,
    tokenize_fn,
    query: str,
    k: int = 50,
    topN_each: int = 500,
    k_rrf: int = 60,
    w_dense: float = 1.0,
    w_bm25: float = 1.0,
    exclude_message_id: str | None = None,
    anchor_date: str | pd.Timestamp | None = None,
    date_col: str = "date_day") -> pd.DataFrame:

    if anchor_date is not None:
        ad = pd.to_datetime(anchor_date, utc=True).normalize()

        if date_col not in df.columns:
            raise KeyError(f"date_col='{date_col}' not found in df.columns")

        dts = pd.to_datetime(df[date_col], errors="coerce", utc=True).dt.normalize()
        allowed = (dts <= ad)
        allowed_np = allowed.to_numpy(dtype=bool)
    else:
        allowed_np = None

    d_idx, d_sc = dense_candidates_faiss(index, encoder, query, topN=topN_each)

    if allowed_np is not None:
        keep = allowed_np[d_idx]
        d_idx = d_idx[keep]
        d_sc = d_sc[keep]

    dense_rank = {int(rowpos): r for r, rowpos in enumerate(d_idx, start=1)}

    if bm25 is None:
        out = df.iloc[d_idx].copy()
        out["_rowpos"] = d_idx
        out["score_rrf"] = (w_dense / (k_rrf + out["_rowpos"].map(lambda rp: dense_rank.get(int(rp), np.nan)).astype(np.float32)))
        out["rank_dense"] = out["_rowpos"].map(lambda rp: dense_rank.get(int(rp), np.nan))
        out["rank_bm25"] = np.nan

        if exclude_message_id is not None and "message_id" in out.columns:
            out = out[out["message_id"].astype(str) != str(exclude_message_id)]

        return out.head(k).reset_index(drop=True)

    bm_scores = bm25.get_scores(tokenize_fn(query)).astype(np.float32)
    if allowed_np is not None:
        bm_scores[~allowed_np] = -np.inf

    b_idx = _topk_indices_from_scores(bm_scores, topN_each)
    bm_rank = {int(rowpos): r for r, rowpos in enumerate(b_idx, start=1)}

    union = np.array(sorted(set(dense_rank) | set(bm_rank)), dtype=int)
    rrf = np.zeros(len(union), dtype=np.float32)

    for j, rowpos in enumerate(union):
        if rowpos in dense_rank:
            rrf[j] += w_dense / (k_rrf + dense_rank[rowpos])
        if rowpos in bm_rank:
            rrf[j] += w_bm25 / (k_rrf + bm_rank[rowpos])

    order = np.argsort(-rrf)
    union = union[order]
    rrf = rrf[order]

    out = df.iloc[union].copy()
    out["_rowpos"] = union
    out["score_rrf"] = rrf
    out["rank_dense"] = out["_rowpos"].map(lambda rp: dense_rank.get(int(rp), np.nan))
    out["rank_bm25"] = out["_rowpos"].map(lambda rp: bm_rank.get(int(rp), np.nan))

    if exclude_message_id is not None and "message_id" in out.columns:
        out = out[out["message_id"].astype(str) != str(exclude_message_id)]

    return out.head(k).reset_index(drop=True)



In [16]:
df["date_day"].min(), df["date_day"].max()

(Timestamp('2023-09-23 00:00:00+0000', tz='UTC'),
 Timestamp('2025-09-08 00:00:00+0000', tz='UTC'))

In [17]:
# ! pip install transformers torch sentence-transformers accelerate vllm

In [18]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import torch

# MODEL = "Qwen/Qwen2.5-14B-Instruct"

# tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL,
#     torch_dtype=torch.bfloat16,
#     device_map="cuda",
#     trust_remote_code=True,
# )
# model.eval()

In [20]:
from transformers import AutoTokenizer, AutoConfig
from vllm import LLM, SamplingParams
import torch

MODEL = "Qwen/Qwen2.5-32B-Instruct"
#MODEL = "mistralai/Mistral-Small-3.1-24B-Instruct-2503"

tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)

cfg = AutoConfig.from_pretrained(MODEL, trust_remote_code=True)
native_ctx = getattr(cfg, "max_position_embeddings", None)
print("GPU:", torch.cuda.get_device_name(0))
print("VRAM(GB):", round(torch.cuda.get_device_properties(0).total_memory / (1024**3), 1))
print("Model max_position_embeddings:", native_ctx)

MAX_MODEL_LEN = 24576


model = LLM(
    model=MODEL,
    dtype="bfloat16",
    max_model_len=MAX_MODEL_LEN,
    gpu_memory_utilization=0.90,
)

GPU: NVIDIA A100-SXM4-80GB
VRAM(GB): 79.3
Model max_position_embeddings: 32768
INFO 01-09 04:47:12 [utils.py:253] non-default args: {'dtype': 'bfloat16', 'max_model_len': 24576, 'disable_log_stats': True, 'model': 'Qwen/Qwen2.5-32B-Instruct'}
INFO 01-09 04:47:13 [model.py:514] Resolved architecture: Qwen2ForCausalLM
INFO 01-09 04:47:13 [model.py:1661] Using max model len 24576
INFO 01-09 04:47:13 [scheduler.py:230] Chunked prefill is enabled with max_num_batched_tokens=8192.
(EngineCore_DP0 pid=11153) INFO 01-09 04:47:20 [core.py:93] Initializing a V1 LLM engine (v0.13.0) with config: model='Qwen/Qwen2.5-32B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-32B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=24576, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=

(EngineCore_DP0 pid=11153) /home/mlcore/conda/lib/python3.10/site-packages/tvm_ffi/_optional_torch_c_dlpack.py:174: UserWarning: Failed to JIT torch c dlpack extension, EnvTensorAllocator will not be enabled.
(EngineCore_DP0 pid=11153) We recommend installing via `pip install torch-c-dlpack-ext`
(EngineCore_DP0 pid=11153)   warnings.warn(


(EngineCore_DP0 pid=11153) INFO 01-09 04:47:25 [cuda.py:351] Using FLASH_ATTN attention backend out of potential backends: ('FLASH_ATTN', 'FLASHINFER', 'TRITON_ATTN', 'FLEX_ATTENTION')


Loading safetensors checkpoint shards:   0% Completed | 0/17 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:   6% Completed | 1/17 [00:00<00:10,  1.59it/s]
Loading safetensors checkpoint shards:  12% Completed | 2/17 [00:01<00:10,  1.43it/s]
Loading safetensors checkpoint shards:  18% Completed | 3/17 [00:02<00:09,  1.43it/s]
Loading safetensors checkpoint shards:  24% Completed | 4/17 [00:02<00:09,  1.44it/s]
Loading safetensors checkpoint shards:  29% Completed | 5/17 [00:03<00:08,  1.46it/s]
Loading safetensors checkpoint shards:  35% Completed | 6/17 [00:04<00:07,  1.48it/s]
Loading safetensors checkpoint shards:  41% Completed | 7/17 [00:04<00:07,  1.41it/s]
Loading safetensors checkpoint shards:  47% Completed | 8/17 [00:05<00:05,  1.53it/s]
Loading safetensors checkpoint shards:  53% Completed | 9/17 [00:06<00:05,  1.55it/s]
Loading safetensors checkpoint shards:  59% Completed | 10/17 [00:06<00:04,  1.51it/s]
Loading safetensors checkpoint shards:  65% Completed | 11/17

(EngineCore_DP0 pid=11153) INFO 01-09 04:47:37 [default_loader.py:308] Loading weights took 11.77 seconds
(EngineCore_DP0 pid=11153) INFO 01-09 04:47:38 [gpu_model_runner.py:3659] Model loading took 61.0375 GiB memory and 15.784764 seconds
(EngineCore_DP0 pid=11153) INFO 01-09 04:47:50 [backends.py:643] Using cache directory: /home/mlcore/.cache/vllm/torch_compile_cache/956b349265/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=11153) INFO 01-09 04:47:50 [backends.py:703] Dynamo bytecode transform time: 11.61 s
(EngineCore_DP0 pid=11153) INFO 01-09 04:48:10 [backends.py:226] Directly load the compiled graph(s) for compile range (1, 8192) from the cache, took 13.591 s
(EngineCore_DP0 pid=11153) INFO 01-09 04:48:10 [monitor.py:34] torch.compile takes 25.20 s in total
(EngineCore_DP0 pid=11153) INFO 01-09 04:48:11 [gpu_worker.py:375] Available KV cache memory: 8.81 GiB
(EngineCore_DP0 pid=11153) INFO 01-09 04:48:11 [kv_cache_utils.py:1291] GPU KV cache size: 36,064 tokens
(

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 51/51 [00:06<00:00,  7.99it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:03<00:00, 10.37it/s]


(EngineCore_DP0 pid=11153) INFO 01-09 04:48:21 [gpu_model_runner.py:4587] Graph capturing finished in 10 secs, took 4.11 GiB
(EngineCore_DP0 pid=11153) INFO 01-09 04:48:22 [core.py:259] init engine (profile, create kv cache, warmup model) took 43.63 seconds
INFO 01-09 04:48:23 [llm.py:360] Supported tasks: ['generate']


In [21]:
SYSTEM_PROMPT = """Ты делаешь аналитический отчёт по новостным фрагментам.

Вход: запрос, актуальная дата (YYYY-MM-DD) и документы вида:
date=YYYY-MM-DD channel=<название>
<текст>

Правила:
- Пиши на русском.
- Опирайся только на предоставленные документы (не добавляй внешние факты).
- Не упоминай номера документов в тексте.
- Если рядом с актуальной датой материалов мало/нет, прямо скажи об этом и не делай выводов “что происходит сейчас”.

Сформируй ответ строго в 4 блоках:

### 1) Запрос и дата
* Запрос: <...>
* Актуальная дата: <...>

### 2) Дайджест
Связное саммари: сначала что было ближе всего к актуальной дате, затем предыстория/волны (если видны).
По возможности отмечай различия в подаче каналов.

### 3) Выводы и актуальность
2–6 буллетов: свежесть (последняя дата и разрыв), плотность (волна/единичные), ширина (сколько каналов), концентрация (доминирует ли один), ограничения данных.

### 4) Таймлайн
Полный список дат из документов, от нового к старому в столбик. Каждая строка - уникальная дата, на которую есть новости:
* YYYY-MM-DD — Канал1; Канал2: кратко что сообщается
"""



def build_rag_context(query: str, cand: pd.DataFrame, anchor_date: str, k_docs: int = 30, snip_chars: int = 850) -> str:
    c = cand.head(k_docs).copy()

    blocks = []
    for i, row in enumerate(c.itertuples(index=False), start=1):
        date_day = getattr(row, "date_day", getattr(row, "date", ""))
        if isinstance(date_day, pd.Timestamp):
            date_day = date_day.strftime("%Y-%m-%d")
        date_day = str(date_day)[:10]

        channel = getattr(row, "channel_name")
        text = getattr(row, "message", "")

        blocks.append(f"[{i}] date={date_day} channel={channel}\n" + snippet(str(text), snip_chars))

    return (
        f"anchor_date: {anchor_date}\n"
        f"ВОПРОС/ЗАПРОС:\n{query}\n\n"
        f"ИСТОЧНИКИ:\n" + "\n\n".join(blocks)
    )



In [22]:
# import torch

# @torch.inference_mode()
# def rag_summarize(sum_model, sum_tokenizer, query: str, cand: pd.DataFrame, anchor_date, 
#                   k_docs: int = 25, snip_chars: int = 900, max_new_tokens: int = 2000) -> str:
    
#     user = build_rag_context(query, cand, anchor_date=anchor_date, k_docs=k_docs, snip_chars=snip_chars)
#     print("built context...")
    
#     messages = [
#         {"role": "system", "content": SYSTEM_PROMPT},
#         {"role": "user", "content": user},
#     ]
#     prompt = sum_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
#     enc = sum_tokenizer(prompt, return_tensors="pt", truncation=True).to(sum_model.device)

#     out_ids = sum_model.generate(
#         **enc,
#         max_new_tokens=max_new_tokens,
#         do_sample=False,
#         eos_token_id=sum_tokenizer.eos_token_id,
#         pad_token_id=sum_tokenizer.eos_token_id,
#     )
#     prompt_len = int(enc["attention_mask"][0].sum().item())
    
#     return sum_tokenizer.decode(out_ids[0][prompt_len:], skip_special_tokens=True).strip(), user


In [23]:
from vllm import SamplingParams

def rag_summarize(sum_model, sum_tokenizer, query: str, cand: pd.DataFrame, anchor_date,
                  k_docs: int = 25, snip_chars: int = 900, max_new_tokens: int = 2000):

    user = build_rag_context(query, cand, anchor_date=anchor_date, k_docs=k_docs, snip_chars=snip_chars)
    print("built context...")

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user},
    ]

    prompt = sum_tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    #Mistral
    #prompt = f"<s>[SYSTEM_PROMPT]{SYSTEM_PROMPT}[/SYSTEM_PROMPT][INST]{user}[/INST]"

    sampling = SamplingParams(
        temperature=0.0,  
        max_tokens=max_new_tokens,
        top_p=1.0,
    )

    result = sum_model.generate([prompt], sampling)[0]
    text = result.outputs[0].text.strip()

    return text, user


In [24]:
def run_rag_hybrid(
    df: pd.DataFrame,
    index,
    encoder,
    bm25,
    tokenize_fn,
    query: str,
    exclude_message_id: str | None = None,
    k_retrieve: int = 50,
    topN_each: int = 500,
    k_docs: int = 25,
    snip_chars: int = 1500,
    max_new_tokens: int = 2000, 
    anchor_date: str = "2025-09-04", 

    sum_model=None,
    sum_tokenizer=None):

    
    cand = hybrid_retrieve_rrf(
        df=df,
        index=index,
        encoder=encoder,
        bm25=bm25,
        tokenize_fn=tokenize_fn,
        query=query,
        k=k_retrieve,
        topN_each=topN_each,
        k_rrf=60,
        w_dense=1.0,
        w_bm25=1.0,
        exclude_message_id=exclude_message_id,
        anchor_date=anchor_date)
    
    if sum_model is None or sum_tokenizer is None:
        ctx = build_rag_context(query, cand, anchor_date=anchor_date, k_docs=k_docs, snip_chars=snip_chars)
        return {"context": ctx, "candidates": cand, "summary": "No LLM"}
    
    summary, ctx = rag_summarize(sum_model, sum_tokenizer, query, cand, k_docs=k_docs, snip_chars=snip_chars, max_new_tokens=max_new_tokens, anchor_date=anchor_date)
    return {"context": ctx, "summary": summary, "candidates": cand}


In [25]:
q = "Актуальный курс доллара"

out = run_rag_hybrid(
    df=df,
    index=index,
    encoder=encoder,
    bm25=bm25,
    tokenize_fn=tokenize_ru,
    query=q,
    k_retrieve=50,
    topN_each=2000,
    k_docs=50,
    snip_chars=1500,
    max_new_tokens = 5000,
    anchor_date = "2025-02-06",
    sum_model=model,           
    sum_tokenizer=tokenizer,
)


built context...


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [26]:
from IPython.display import display, Markdown, HTML
import re

def show_summary(summary: str):
    if summary is None:
        display(HTML("<b>summary is None</b>"))
        return

    s = str(summary)
    s = s.replace("\\n", "\n") 
    s = re.sub(r"\n{3,}", "\n\n", s).strip()

    display(Markdown(s))

show_summary(out["summary"])


### 1) Запрос и дата
* Запрос: Актуальный курс доллара
* Актуальная дата: 2025-02-06

### 2) Дайджест
На 3 февраля 2025 года ЦБ поднял курс доллара более чем на 2 рубля, приблизив его к 100 рублям. В предыдущие дни курс доллара колебался, но в целом демонстрировал тенденцию к росту. Например, 23 января курс доллара был снижён до 98,2804 рубля, а 21 января — до 101,9579 рубля. В конце декабря 2024 года курс доллара также снижался, достигнув 99,22 рубля за доллар. В середине декабря курс доллара был повышен до 103,3 рубля, а затем снова понижен до 99,3759 рубля. В ноябре курс доллара достигал 114 рублей, но затем начал снижаться. В начале декабря курс доллара был снижён до 99 рублей. В середине ноября курс доллара достигал 111 рублей, а затем продолжал расти до 114 рублей. В начале ноября курс доллара был повышен до 102 рублей. В октябре курс доллара колебался около 96 рублей. В июне курс доллара был около 85 рублей.

### 3) Выводы и актуальность
- **Свежесть**: Последняя дата — 3 февраля 2025 года, разрыв до актуальной даты составляет 3 дня.
- **Плотность**: Новости о курсе доллара поступали регулярно, но в последние дни активность снизилась.
- **Ширина**: Новости о курсе доллара публиковались на нескольких каналах, включая Блумберг, Банки, деньги, два офшора, Сигналы РЦБ и другие.
- **Концентрация**: Основные новости о курсе доллара публиковались на каналах Банки, деньги, два офшора и Сигналы РЦБ.
- **Ограничения данных**: Новости о курсе доллара в последние дни были немногочисленны, что затрудняет анализ текущей ситуации.

### 4) Таймлайн
* 2025-02-03 — Блумберг: Курс доллара от ЦБ вырос более чем на 2 рубля, вплотную приблизившись к 100.
* 2025-02-03 — Банки, деньги, два офшора: ЦБ поднял курс доллара сразу на два рубля, почти до 100 рублей.
* 2025-01-29 — Сигналы РЦБ: ЦБ РФ повысил официальный курс доллара на 4,68 коп., до 98,0126 руб.
* 2025-01-24 — Сигналы РЦБ: ЦБ РФ снизил официальный курс доллара на 83,42 коп., до 98,2636 руб.
* 2025-01-22 — Сигналы РЦБ: ЦБ РФ снизил официальный курс доллара на 1,65 руб., до 98,2804 руб.
* 2025-01-20 — Сигналы РЦБ: ЦБ РФ снизил официальный курс доллара на 45,58 коп., до 101,9579 руб.
* 2024-12-31 — Раньше всех. Ну почти: Курс доллара на внебиржевых торгах почти превысил 114 рублей.
* 2024-12-26 — Блумберг: Рубль продолжает укрепляться — ЦБ установил официальный курс в 99,22 рубля за доллар.
* 2024-12-24 — Сигналы РЦБ: ЦБ РФ снизил официальный курс доллара на 1,74 руб., до 99,8729 руб.
* 2024-12-24 — Банки, деньги, два офшора: Официальный курс доллара упал ниже 100.
* 2024-12-23 — Сигналы РЦБ: ЦБ РФ снизил официальный курс доллара на 72,95 коп., до 101,6143 руб.
* 2024-12-20 — Раньше всех. Ну почти: Официальный курс доллара составляет 102,3438 руб./$1.
* 2024-12-14 — Банки, деньги, два офшора: Экономически обоснованный курс доллара — 100-110 рублей.
* 2024-12-11 — Банки, деньги, два офшора: ЦБ поднял курс доллара выше 103 рублей, евро — выше 108,5 рубля.
* 2024-12-10 — Банки, деньги, два офшора: ЦБ снова поднял курс доллара выше 100 рублей, евро — выше 106.
* 2024-12-09 — Сигналы РЦБ: ЦБ РФ снизил официальный курс доллара на 4,56 коп., до 99,3759 руб.
* 2024-12-06 — Банки, деньги, два офшора: ЦБ понизил курс доллара ниже 100, евро — ниже 107.
* 2024-12-06 — Блумберг: ЦБ понизил официальный курс доллара до 99 рублей.
* 2024-12-05 — Раньше всех. Ну почти: Банк России установил на 6 декабря официальный курс доллара на уровне 103,3837 руб.
* 2024-12-05 — Банки, деньги, два офшора: ЦБ установил новые курсы для доллара и евро — 103,3 и 109,7 соответственно.
* 2024-12-03 — Раньше всех. Ну почти: Официальный курс доллара составляет 106,1878 руб./$1.
* 2024-11-28 — Блумберг: Официальный курс доллара от ЦБ превысил 109 рублей.
* 2024-11-28 — Блумберг: Курс доллара опустился ниже отметки 110 рублей.
* 2024-11-27 — Банки, деньги, два офшора: Доллар — 114, евро — 120.
* 2024-11-27 — Блумберг: Курс доллара уже выше 110 рублей и продолжает расти.
* 2024-11-27 — Блумберг: Курс доллара перевалил за 111 рублей. UPD: Курс уже 114.
* 2024-11-27 — Сигналы РЦБ: ЦБ снизил официальный курс доллара на 1,84 руб., до 107,7409 руб.
* 2024-11-26 — Банки, деньги, два офшора: ЦБ поднял официальный курс доллара до 105 рублей, евро — до 110 рублей.
* 2024-11-25 — Банки, деньги, два офшора: Курс доллара достиг почти 104 рублей по оценке ЦБ.
* 2024-11-22 — Forbes Russia: ЦБ установил официальный курс доллара выше 102 рублей впервые с марта 2022 года.
* 2024-11-22 — Банки, деньги, два офшора: ЦБ резко повысил курс доллара до 102 рублей, евро — до 107 рублей.
* 2024-11-19 — Банки, деньги, два офшора: Официальный курс доллара превысил 100 рублей.
* 2024-11-15 — Банки, деньги, два офшора: ЦБ поднял курс доллара почти до 100 рублей.
* 2024-11-14 — Экономика: Курс доллара теперь стоит выше 100 рублей.
* 2024-11-09 — Банки, деньги, два офшора: Курс доллара достиг 100 рублей в моменте на Forex.
* 2024-10-11 — Сигналы РЦБ: ЦБ РФ снизил официальный курс доллара на 1,17 руб., до 96,0686 руб.
* 2024-10-07 — Сигналы РЦБ: Официальный курс доллара на максимуме с 20 октября 2023 года.
* 2024-10-01 — Экономика: Центральный банк России установил официальный курс доллара на уровне 93,2221 рубля.
* 2024-10-01 — Сигналы РЦБ: Курс доллара на бирже ICE достигает почти 96 руб.
* 2024-06-27 — Блумберг: ЦБ РФ понизил курс доллара до 84,96 руб., евро — до 90,99 руб., юаня — до 11,48 руб.
* 2024-06-20 — Блумберг: ЦБ повысил курс доллара до 85,42 рубля, евро — до 91,45 рубля, юаня — до 11,52 рубля.
* 2024-06-13 — Блумберг: ЦБ после санкций США в отношении Мосбиржи установил курс доллара на уровне 88,21 рубля.
* 2024-06-13 — Блумберг: Курс доллара упал до 87₽.
* 2024-06-12 — Блумберг: Курс доллара в криптообменниках поднялся до 96 рублей.
* 2024-04-30 — Блумберг: Курс доллара упал ниже 93₽.
* 2024-04-26 — Блумберг: Актуальные курсы валют: доллар – 87.87₽, евро – 94.1₽.
* 2024-04-19 — Блумберг: Курс доллара может достигнуть 150-160 рублей.

In [ ]:
out["context"]

In [ ]:
import torch

props = torch.cuda.get_device_properties(0)
vram_gb = props.total_memory / (1024**3)
print("GPU:", props.name)
print("VRAM (GB):", round(vram_gb, 1))

In [ ]:
Давай попробуем отфильтровать маленькой моделькой JUDGE_MODEL нерелевантные документы из топ-к и поменять промпт так, чтобы, если в контексте не останется релевантных документов, мы явно скажем, что новостей об этом нет, но вот что можно сказать об этой ситуации (ллм скажет от себя)

ну и будем делать более глубокий ретрив